In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torch.utils.data import DataLoader, Dataset   
import os
from transformers import  AutoModelForCausalLM, AutoTokenizer
import torch  
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments
from transformers import Trainer
from transformers import pipeline
from accelerate import DataLoaderConfiguration 
from transformers import Trainer


In [2]:
source = "intents.txt"

In [3]:

model = AutoModelForCausalLM.from_pretrained("Writer/palmyra-small")

tokenizer = AutoTokenizer.from_pretrained("Writer/palmyra-small")


Widget Javascript not detected.  It may not be installed or enabled properly.


AttributeError: 'FloatProgress' object has no attribute 'style'

In [4]:
if tokenizer.eos_token:
    tokenizer.pad_token = tokenizer.eos_token
else:
    # Add a new padding token and resize model embeddings
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# Confirm padding token set
print("Tokenizer padding token: ", tokenizer.pad_token)

Tokenizer padding token:  <|endoftext|>


In [5]:
from transformers import DataCollatorWithPadding
data = []
with open('/kaggle/input/intents/intents.txt', 'r', encoding='utf-8') as file:
    question, answer = None, None
    for line in file:
        if line.startswith('question :'):
            if question is not None and answer is not None:
                # Save the previous Q&A pair before moving to the next question
                data.append({'question': question, 'answer': answer})
            # Extract the question part and strip leading and trailing whitespaces
            question = line[len('question :'):].strip()
        elif line.startswith('answer :'):
            # Extract the answer part and strip leading and trailing whitespaces
            answer = line[len('answer :'):].strip()
    
    # Don't forget to add the last Q&A pair
    if question is not None and answer is not None:
        data.append({'question': question, 'answer': answer})

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)
max_length=512
tokenized_data = df.apply(
    lambda row: tokenizer(
        row['question'], 
        row['answer'],
        max_length=max_length,  # enforce the max_length
        padding='max_length',   # pad all sequences to the max_length
        truncation=True         # truncate sequences longer than max_length
    ), 
    axis=1
)

df_tokenized = pd.DataFrame({
    'input_ids': tokenized_data.apply(lambda x: x['input_ids']),
    'attention_mask': tokenized_data.apply(lambda x: x['attention_mask']),
    'token_type_ids': tokenized_data.apply(lambda x: x['token_type_ids']) if 'token_type_ids' in tokenized_data.iloc[0] else None,
})
class CustomDataset(Dataset):
    def __init__(self, tokenizer, dataframe, max_length=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        inputs = self.tokenizer.encode_plus(
            row['question'],
            row['answer'],
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = inputs['input_ids'].squeeze()
        labels = input_ids.clone()
        labels[:-1] = input_ids[1:]
        labels[-1] = -100  # Ignore the last token for loss calculation

        return {
            'input_ids': input_ids,
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': labels
        }

# Split data into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.1)

# Create instances of the dataset
train_dataset = CustomDataset(tokenizer, train_df, max_length=512)
val_dataset = CustomDataset(tokenizer, val_df, max_length=512)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [6]:
print(df.head())

                            question  \
0                What to do if Cuts?   
1                  How to cure Cuts?   
2  Which medicine to apply for Cuts?   
3             what to apply on cuts?   
4                               Cuts   

                                              answer  
0  Wash the cut properly to prevent infection and...  
1  Wash the cut properly to prevent infection and...  
2  Wash the cut properly to prevent infection and...  
3  Wash the cut properly to prevent infection and...  
4  Wash the cut properly to prevent infection and...  


In [7]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3
)

In [8]:
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last layer parameters
for name, param in model.named_parameters():
    if 'h.11' in name or 'transformer.ln_f.bias' in name or 'transformer.ln_f.weight' in name:
        param.requires_grad = True

# Check which parameters are trainable
for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")

transformer.wte.weight: False
transformer.wpe.weight: False
transformer.h.0.ln_1.weight: False
transformer.h.0.ln_1.bias: False
transformer.h.0.attn.c_attn.weight: False
transformer.h.0.attn.c_attn.bias: False
transformer.h.0.attn.c_proj.weight: False
transformer.h.0.attn.c_proj.bias: False
transformer.h.0.ln_2.weight: False
transformer.h.0.ln_2.bias: False
transformer.h.0.mlp.c_fc.weight: False
transformer.h.0.mlp.c_fc.bias: False
transformer.h.0.mlp.c_proj.weight: False
transformer.h.0.mlp.c_proj.bias: False
transformer.h.1.ln_1.weight: False
transformer.h.1.ln_1.bias: False
transformer.h.1.attn.c_attn.weight: False
transformer.h.1.attn.c_attn.bias: False
transformer.h.1.attn.c_proj.weight: False
transformer.h.1.attn.c_proj.bias: False
transformer.h.1.ln_2.weight: False
transformer.h.1.ln_2.bias: False
transformer.h.1.mlp.c_fc.weight: False
transformer.h.1.mlp.c_fc.bias: False
transformer.h.1.mlp.c_proj.weight: False
transformer.h.1.mlp.c_proj.bias: False
transformer.h.2.ln_1.weight:

In [9]:
print(train_dataset[0])


{'input_ids': tensor([48979,  4831,   329,   257,  1744,  3211, 31846,    30,  5211,   407,
         1445,   262,  3211,    26, 32411,   262,  3211,   287,   663,  1459,
         2292,    13,  1002,  1695,    11,   779,   257,  4328,   600,   393,
        46704,   284, 47800,  1096,   262,  3211,   475,   466,   407,  1949,
          284,  1103,   570,   340,    13, 27967,  4771, 12908,   287,   257,
        16270,   284,  4646, 29844,    13, 24390,  3501,  2356, 17638,   878,
         1972,  3315,  5608,    13, 40749,  4708,  3315,  1037,  3393,    13,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 

In [10]:
trainer =Trainer(
        model=model,
    
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [11]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer , truncation=True )

generated_text = generator("Example input text", max_length=50)
print(generated_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Example input text\n                                              '}]


### Training

In [12]:
trainer.train() 

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


RuntimeError: module must have its parameters and buffers on device cuda:0 (device_ids[0]) but found one of them on device: cpu

In [ ]:
torch.save(model.state_dict(), '/kaggle/working/model_weights.pth')

In [ ]:
model.load_state_dict(torch.load('model_weights.pth'))
model.eval() 

In [ ]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

generated_text = generator("cut", max_length=50)
print(generated_text)